In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms, models              # For image transformations and pretrained models

In [2]:
class SiameseNetwork(nn.Module):
    def __init__(self, embedding_dim=128):
        super().__init__()
        # Load a pre-trained ResNet18 model
        backbone = models.resnet18(pretrained=True)
        modules = list(backbone.children())[:-1]  # Remove the final classification layer
        self.feature_extractor = nn.Sequential(*modules)  # Use everything except the final FC layer

        # Add a new fully connected layer to project features into a lower-dimensional embedding space
        self.fc = nn.Linear(backbone.fc.in_features, embedding_dim)

    def forward_once(self, x):
        # Pass input through the feature extractor
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.fc(x)             # Project to embedding_dim
        return F.normalize(x, p=2, dim=1)  # Normalize embeddings to have unit L2 norm

    def forward(self, datapoint):
        # Forward pass for a triplet: anchor (a), positive (p), negative (n)
        return self.forward_once(datapoint)

transform = transforms.Compose([
    transforms.Resize((256, 256)),              # Resize images to 256x256
    transforms.CenterCrop(224),                 # Crop the center 224x224 patch (standard size for ResNet)
    transforms.ToTensor(),                      # Convert PIL images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize using ImageNet statistics
                         std=[0.229, 0.224, 0.225]),
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork().to(device)

/home/cognidev-technologies/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/cognidev-technologies/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
### Cell 8: Save and Load Model
save_path = "triplet_siamese.pth"
loaded_model = SiameseNetwork().to(device)
loaded_model.load_state_dict(torch.load(save_path, map_location=device))
loaded_model.eval()
print("Model loaded and set to eval mode.")

Model loaded and set to eval mode.


In [5]:
# Load an image and apply transformations
image_path = "img.jpg"  # Replace with your image path
image = Image.open(image_path).convert("RGB")  # Open image and convert to RGB
image = transform(image).unsqueeze(0).to(device)  # Apply transformations and add batch dimension
with torch.no_grad():
    embedding = loaded_model(image)  # Get the embedding for the image
    print("Image embedding shape:", embedding.shape)  # Print the shape of the embedding
    print("Image embedding:", embedding)  # Print the actual embedding values

Image embedding shape: torch.Size([1, 128])
Image embedding: tensor([[-0.0532,  0.0547,  0.0531, -0.0663, -0.0696, -0.1357, -0.2329,  0.1162,
         -0.0092,  0.0008, -0.1556, -0.0870,  0.0632, -0.0354, -0.1561,  0.1169,
          0.1454, -0.1705,  0.1526, -0.0329, -0.1132,  0.0993,  0.0466, -0.0825,
          0.0098,  0.1143, -0.0656, -0.0241, -0.0100,  0.0940, -0.0042, -0.0191,
          0.0286,  0.0128,  0.0050, -0.0791, -0.1762,  0.0976,  0.2020, -0.1029,
         -0.0150, -0.0396, -0.0533,  0.1395,  0.1268,  0.0084, -0.1241, -0.0403,
         -0.0827, -0.1633, -0.0154,  0.0653, -0.0143,  0.1328, -0.1496,  0.1572,
         -0.1106, -0.1461, -0.0201, -0.0721, -0.0790,  0.1073, -0.0568, -0.0821,
          0.0321,  0.0427,  0.0141,  0.0927, -0.0192, -0.0739,  0.2465,  0.1379,
          0.0482, -0.0021, -0.0247, -0.0036,  0.0603, -0.1163, -0.0420,  0.0072,
          0.0103, -0.0015, -0.0183, -0.1523,  0.0496,  0.0026, -0.0440, -0.0309,
          0.0766, -0.0734, -0.0992, -0.0824,  0.